In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz

pd.set_option("display.max_columns", None)

path = "C:/Users/Admin/Documents/ironhack/streaming_service_recommender/"

# User Data Frame

#### Goals

- Create function to read newest input survey and transform it to the same format as genres_recommender in order to append it and get the most similar streaming service.

Survey Link: https://forms.gle/Jb7gzK7VMbnvpRvP9

### 1. Import data

In [2]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"

survey = pd.read_csv(url)

In [3]:
survey

,Timestamp,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Quesiton 10,Question 11,Question 12,Question 13,Question 14,Question 15,Question 16,Question 17,Question 18,Quesiton 20,Question 19,Question 21,Question 22,Question 23,Question 24,Question 25,Question 26,Question 27,Question 28,Question 29,Question 30
0,5/14/2020 12:52:36,Revisions (2019) - When Shibuya time-warps to ...,Big Mouth (2017) - Teenage friends find their ...,American Horror Story (2011) - An anthology se...,Chilling Adventures of Sabrina (2018) - As her...,Hart of Dixie (2011) - New Yorker and new doct...,Black Mirror (2011) - An anthology series expl...,Stranger Things (2016) - When a young boy disa...,Crazy Ex-Girlfriend (2015) - A young woman aba...,Supernatural (2005) - Two brothers follow thei...,Fauda (2015) - The human stories on both sides...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5/14/2020 13:36:32,Survivor (2000) - A reality show where a group...,The Amazing Race (2001) - Multiple teams race ...,Klondike (2014) - The lives of two childhood b...,The Planets (2019) - Welcome to the greatest s...,The Joy of Painting (1983) - In this half-hour...,Eat. Race. Win (2018) - EAT. RACE. WIN. is a b...,Generation Kill (2008) - A Rolling Stone repor...,Bonanza (1959) - The Wild West adventures of B...,How The Universe Works (2010) - A users guide ...,Strictly Ballet (2014) - Join us as we meet Ja...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5/14/2020 13:36:53,John Adams (2008) -The life of one of the USA'...,Sharp Objects (2018) - A reporter confronts th...,The Sopranos (1999) - New Jersey mob boss Tony...,Vinyl (2016) - A New York music executive in t...,Ballers (2015) - A series centered around a gr...,Any Given Wednesday with Bill Simmons (2016) -...,Silicon Valley (2014) - Follows the struggle o...,The Night Of (2016) - After a night of partyin...,Last Week Tonight with John Oliver (2014) - Fo...,VICE News Tonight (2016) - A nightly news show...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5/14/2020 17:10:17,Revisions (2019) - When Shibuya time-warps to ...,Big Mouth (2017) - Teenage friends find their ...,American Horror Story (2011) - An anthology se...,Chilling Adventures of Sabrina (2018) - As her...,Hart of Dixie (2011) - New Yorker and new doct...,Black Mirror (2011) - An anthology series expl...,Stranger Things (2016) - When a young boy disa...,Crazy Ex-Girlfriend (2015) - A young woman aba...,Supernatural (2005) - Two brothers follow thei...,Fauda (2015) - The human stories on both sides...,The Witcher () -,Cheer () -,Ozark () -,"Tiger King: Murder, Mayhem and Madness () -",Yeh Meri Family () -,Five Came Back () -,The Circle () -,Rythm + Flow () -,The Motorbike Show () -,The 100 () -,The Stranger () -,Blue Mountain State () -,The Curious Creations of Christine McConnell () -,The Vietnam War () -,The Ranch () -,The End of the F***ing World () -,Once Upon a Time () -,The Flash () -,BoJack Horseman () -,"Love, Death & Robots () -"


We have three rows which were used for testing, first row is selection of Netflix's TV shows, second is Amazon's, and third is HBO's. We will create an individual data frame for each in order to test our model, we will do first Netflix's data frame and afterwards, create a function to get Amazon's and HBO's.

We will also need to join all streaming services genres data frames in order to obtain the genres data frame for our survey users.

In [4]:
netflix = pd.read_pickle(path + "Data/netflix_genres.pkl")
amazon = pd.read_pickle(path + "Data/amazon_genres.pkl")
hbo = pd.read_pickle(path + "Data/hbo_genres.pkl")

In [5]:
all_shows = netflix.append(amazon).append(hbo).reset_index(drop=True)

In [6]:
# all_shows.to_pickle(path + "Data/all_shows_genres.pkl")

### 2. Remove Timestamp column

We will remove the Timestamp column since it would not be useful for our analysis.

In [7]:
survey = survey.drop(columns="Timestamp")

### 3. Get list of TV shows for survey user

In [8]:
all_shows_lst = [show for show in all_shows["show"]]

In [25]:
for i in survey.loc[0].values:
    print(i)

Revisions (2019) - When Shibuya time-warps to 2388, high school student Daisuke and his friends are conscripted by AHRV agent Milo to fight revisions, a hostile cyborg race.
Big Mouth (2017) - Teenage friends find their lives upended by the wonders and horrors of puberty.
American Horror Story (2011) - An anthology series centering on different characters and locations, including a house with a murderous past, an insane asylum, a witch coven, a freak show circus, a haunted hotel, a possessed farmhouse, a cult, the apocalypse, and a slasher summer camp.
Chilling Adventures of Sabrina (2018) - As her 16th birthday nears, Sabrina must choose between the witch world of her family and the human world of her friends. Based on the Archie comic.
Hart of Dixie (2011) - New Yorker and new doctor Zoe Hart accepts an offer from a stranger, Dr. Harley Wilkes, to work in his medical practice in Bluebell, Alabama. She arrives to find he has died and left half the practice to her in his will.
Black Mi

In [24]:
# using regex, we will get the title of TV shows selected

titles_netflix = []

for show in survey.loc[0].values:
    # remove the year and description
    title_pattern = r"(.*)\s\(\d{4}\).*$"
    title = re.findall(title_pattern, show)
    titles_netflix.append(title[0])


matches_netflix = []

for title in titles_netflix:
    for show in all_shows_lst:
        
        # compute ratio and remove case-sensitivity
        ratio = fuzz.ratio(title.lower(), show.lower())

        # add titles to list with ratio = 100
        if ratio == 100:
            matches_netflix.append(show)
            
            # we will remove duplicates since there might be shared TV shows between streaming services
            matches_netflix = list(set(matches_netflix))

matches_netflix

TypeError: expected string or bytes-like object

It works, now we will get the data frame for the shows listed on matches_netflix.

### 4. Create data frame to use for recommender model

In [10]:
user_netflix = all_shows[all_shows["show"].isin(matches_netflix)].reset_index(drop=True)

user_netflix

,show,genres,Crime,Drama,Thriller,Fantasy,Horror,Mystery,Comedy,Sci-Fi,Biography,Action,Adventure,Romance,History,Documentary,Animation,War,Sport,Family,Western,Short,Reality-TV,Musical,Music,Game-Show,Talk-Show,News
0,Stranger Things,"Drama,Fantasy,Horror",0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
1,Black Mirror,"Drama,Sci-Fi,Thriller",0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
2,Supernatural,"Drama,Fantasy,Horror",0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
3,Big Mouth,"Animation,Comedy,Romance",0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0.0,0.0,0,0,0,0,0,0
4,American Horror Story,"Drama,Horror,Thriller",0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
5,Fauda,"Action,Drama,Thriller",0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
6,Chilling Adventures of Sabrina,"Drama,Fantasy,Horror",0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
7,Hart of Dixie,"Comedy,Drama,Romance",0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
8,Crazy Ex-Girlfriend,"Comedy,Musical",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1,1,0,0,0
9,revisions,"Action,Animation,Sci-Fi",0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0.0,0.0,0,0,0,0,0,0


We will follow the same process as done on 05_genres and 06.1_streaming_service_recommender_df

In [11]:
genre_columns = [col for col in user_netflix.columns if col not in ["show", "genres"]]

In [12]:
user_netflix_ratio = (pd.DataFrame(user_netflix[genre_columns].sum()
                                    .sort_values(ascending=False))
                       .reset_index()
                       .rename(columns={"index":"genre", 0:"frequency"}))

user_netflix_ratio["ratio"] = round((user_netflix_ratio["frequency"] / 10), 4)

user_netflix_ratio["user"] = "User_Netflix"

user_netflix_recommender = user_netflix_ratio.pivot_table(index="user",
                                      columns="genre",
                                      values="ratio")

In [13]:
user_netflix_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,,,,,,
User_Netflix,0.2,0.0,0.2,0.0,0.3,0.0,0.0,0.8,0.0,0.3,0.0,0.0,0.5,0.1,0.1,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.4,0.0,0.0


Now we have our final dataframe, we will now create a function that sums up all of this steps.

### 5. Create function

In [14]:
# the input of this function will be the df survey, all_shows df, genre_columns and row location

def get_recommender_df(survey_df, df_all_shows, genre_columns, row):
    
    survey_df = survey_df.drop(columns="Timestamp")
    
    all_shows_lst = [show for show in df_all_shows["show"]]
    
    titles = []

    for show in survey_df.loc[row].values:
        # remove the year and description
        title_pattern = r"(.*)\s\(\d{4}\).*$"
        title = re.findall(title_pattern, show)
        titles.append(title[0])

    matches = []

    for title in titles:
        for show in all_shows_lst:
            ratio = fuzz.ratio(title.lower(), show.lower())
            if ratio == 100:
                matches.append(show)
                matches = list(set(matches))
                              
    user_df = df_all_shows[df_all_shows["show"].isin(matches)].reset_index(drop=True)
    
    user_genre_ratio = (pd.DataFrame(user_df[genre_columns].sum()
                                    .sort_values(ascending=False))
                       .reset_index()
                       .rename(columns={"index":"genre", 0:"frequency"}))

    user_genre_ratio["ratio"] = round((user_genre_ratio["frequency"] / 10), 4)

    user_genre_ratio["user"] = "User"

    user_recommender = user_genre_ratio.pivot_table(index="user",
                                          columns="genre",
                                          values="ratio")
    
    return user_recommender

In order to test the function, we will import everything needed again.

In [15]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"

survey_test = pd.read_csv(url)

all_shows_test = pd.read_pickle(path + "Data/all_shows_genres.pkl")

genre_columns = [col for col in all_shows_test.columns if col not in ["show", "genres"]]

In [16]:
user_amazon_recommender = get_recommender_df(survey_test, all_shows_test, genre_columns, 1)

In [17]:
user_amazon_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,,,,,,
User,0.0,0.3,0.0,0.0,0.0,0.0,0.4,0.3,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.1,0.0,0.0,0.0,0.2,0.2


In [18]:
user_hbo_recommender = get_recommender_df(survey_test, all_shows_test, genre_columns, 2)

In [19]:
user_hbo_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,,,,,,
User,0.0,0.0,0.0,0.2,0.3,0.4,0.0,0.8,0.0,0.0,0.0,0.2,0.0,0.1,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0


It works, now we will export our data to test our model.

### 6. Export data

In [20]:
# user_netflix_recommender.to_pickle(path + "Data/netflix_test.pkl")

# user_amazon_recommender.to_pickle(path + "Data/amazon_test.pkl")

# user_hbo_recommender.to_pickle(path + "Data/hbo_test.pkl")